In [1]:
import openai
import langchain
import pinecone 
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
# from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import json

/Users/subratomondal/miniconda3/envs/langchain_demo_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# github = https://github.com/krishnaik06/Complete-Langchain-Tutorials/tree/main/LLM%20Generic%20APP
# youtube = https://www.youtube.com/watch?v=erUfLIi9OFM&t=1809s

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os

In [5]:
## Lets Read the document
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [6]:
doc = read_doc('docs/') ## Page wise docs
print('Loaded Docs: ', len(doc))

Loaded Docs:  58


In [7]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [8]:
documents=chunk_data(docs=doc)
len(documents)

58

In [9]:
documents[2:4]

[Document(metadata={'producer': 'Adobe Acrobat Pro 10.1.16', 'creator': 'Adobe Acrobat Pro 10.1.16', 'creationdate': '2023-02-01T05:28:04+05:30', 'moddate': '2023-02-01T08:28:21+05:30', 'title': '', 'source': 'docs/budget_speech.pdf', 'total_pages': 58, 'page': 2, 'page_label': '3'}, page_content='CONTENTS \nPART-A \n Page No. \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vision for Amrit Kaal – an empowered and inclusive economy 3 \n\uf0b7 Priorities of this Budget 5 \ni. Inclusive Development  \nii. Reaching the Last Mile \niii. Infrastructure and Investment \niv. Unleashing the Potential \nv. Green Growth \nvi. Youth Power  \nvii. Financial Sector \n \n \n \n \n \n \n \n \n \n\uf0b7 Fiscal Management \n24 \nPART B \n  \nIndirect Taxes 27 \n\uf0b7 Green Mobility  \n\uf0b7 Electronics   \n\uf0b7 Electrical   \n\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Marine products  \n\uf0b7 Lab Grown Diamonds  \n\uf0b7 Precious Metals  \n\uf0b7 Meta

In [10]:
MISTRAL_API_KEY=os.environ['MISTRAL_API_KEY']
print(MISTRAL_API_KEY)

ZjEeDE9XK7vad3MWa2QgpKBhsCeZrPsx


In [11]:
## Embedding Technique Of OPENAI
"""
api_key=os.environ['OPENAI_API_KEY']
print(api_key)

pinecone_api_key=os.environ['PINECONE_API_KEY']
print(pinecone_api_key)

embeddings=OpenAIEmbeddings(api_key=api_key)
embeddings

# Initialize a Pinecone client with your API key
from pinecone import Pinecone, ServerlessSpec
import getpass

# if not os.getenv("PINECONE_API_KEY"):
#    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pc = Pinecone(api_key=pinecone_api_key)

import time

index_name = "langchain-test-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
index.list

from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)
vector_store.add_documents(documents=documents, ids=uuids)

llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")
"""

'\napi_key=os.environ[\'OPENAI_API_KEY\']\nprint(api_key)\n\npinecone_api_key=os.environ[\'PINECONE_API_KEY\']\nprint(pinecone_api_key)\n\nembeddings=OpenAIEmbeddings(api_key=api_key)\nembeddings\n\n# Initialize a Pinecone client with your API key\nfrom pinecone import Pinecone, ServerlessSpec\nimport getpass\n\n# if not os.getenv("PINECONE_API_KEY"):\n#    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")\n\npc = Pinecone(api_key=pinecone_api_key)\n\nimport time\n\nindex_name = "langchain-test-index"  # change if desired\n\nexisting_indexes = [index_info["name"] for index_info in pc.list_indexes()]\n\nif index_name not in existing_indexes:\n    pc.create_index(\n        name=index_name,\n        dimension=3072,\n        metric="cosine",\n        spec=ServerlessSpec(cloud="aws", region="us-east-1"),\n    )\n    while not pc.describe_index(index_name).status["ready"]:\n        time.sleep(1)\n\nindex = pc.Index(index_name)\nindex.list\n\nfrom langchain_pin

In [12]:
from mistralai import Mistral

mistral_model = "mistral-embed"

mistral_client = Mistral(api_key=MISTRAL_API_KEY)

embeddings_batch_response = mistral_client.embeddings.create(
    model=mistral_model,
    inputs=["Embed this sentence.", "As well as this one."],
)

print(embeddings_batch_response)
TOKEN_LENGTH = len(embeddings_batch_response.data[0].embedding)
print('Token Len:', TOKEN_LENGTH)

id='bbf96bdfe0b44da9a726690b7bc7b624' object='list' model='mistral-embed' usage=UsageInfo(prompt_tokens=15, completion_tokens=0, total_tokens=15) data=[EmbeddingResponseData(object='embedding', embedding=[-0.016632080078125, 0.0701904296875, 0.03143310546875, 0.01309967041015625, 0.0202789306640625, 0.009735107421875, 0.0255126953125, 0.0018939971923828125, -0.0087738037109375, -0.0087738037109375, -0.039642333984375, 0.058319091796875, -0.025634765625, 0.0074615478515625, -0.0290374755859375, 0.040313720703125, 0.0546875, 0.0257415771484375, 0.03167724609375, 0.02288818359375, -0.056732177734375, -0.01617431640625, -0.061492919921875, 0.01241302490234375, -0.0457763671875, -0.026763916015625, -0.00797271728515625, -0.037811279296875, -0.03985595703125, 0.0009074211120605469, 0.02392578125, -0.0300750732421875, 0.0305328369140625, -0.0024776458740234375, -0.0125274658203125, -0.036224365234375, -0.03326416015625, -0.044891357421875, 0.01355743408203125, 0.0019216537475585938, 0.0099105

In [1]:
from sentence_transformers import SentenceTransformer

st_model = SentenceTransformer("nvidia/NV-Embed-v2", trust_remote_code=True)

sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium."
]
embeddings = st_model.encode(sentences)

print(embeddings)
# [3, 3]

/Users/subratomondal/miniconda3/envs/langchain_demo_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:36<00:00,  9.05s/it]


RuntimeError: MPS backend out of memory (MPS allocated: 20.40 GB, other allocations: 464.00 KB, max allowed: 20.40 GB). Tried to allocate 16.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [13]:
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="llama3"
)

In [14]:
vector = embed.embed_query("How are you?")
print("Vactor len:", len(vector))
print()

Vactor len: 4096



In [15]:
documents[2:4]
print("Len:", len(documents), type(documents[0]))
doc_subset = documents[:10]
print("Len doc_subset:", len(doc_subset))

Len: 58 <class 'langchain_core.documents.base.Document'>
Len doc_subset: 10


In [16]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents))]

In [17]:
data = [
  { 
    "id":uuids[i], 
    "metadata": documents[i].metadata, 
    #"vector": embed.embed_query(documents[i].page_content),
    "vector": mistral_client.embeddings.create( model=mistral_model, inputs=[documents[i].page_content],).data[0].embedding,
    "content": documents[i].page_content 
  } for i in range(len(documents))
]

SDKError: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}

In [ ]:
print(json.dumps(data[0], indent=4, sort_keys=True))

In [ ]:

print(json.dumps(data[0], indent=4, sort_keys=True))
print(len(data[0]['vector']))
print(len(data[0]['id']))
max_len = 0
for doc in data:
    if len(doc["content"]) > max_len:
        max_len = len(doc["content"])
        
print('max_len:', max_len)

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

In [ ]:
"""
client.create_database(
    db_name="my_database_1"
)
"""

In [ ]:
from pymilvus import DataType

schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

# 2.2. Add fields to schema
schema.add_field(field_name="id", datatype=DataType.VARCHAR, max_length=36, is_primary=True)
schema.add_field(field_name="content", datatype=DataType.VARCHAR, max_length=(max_len+200))
schema.add_field(field_name="metadata", datatype=DataType.JSON)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=TOKEN_LENGTH)


In [ ]:
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="vector",
    index_name="vector_index",
    index_type="FLAT",
    metric_type="COSINE",
    params={}
)

In [ ]:
COLLECTION_NAME = "demo_collection"

In [ ]:
if client.has_collection(collection_name=COLLECTION_NAME):
    client.drop_collection(collection_name=COLLECTION_NAME)
    
client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=4096,  # The vectors we will use in this demo has 768 dimensions
    schema=schema,
    index_params=index_params
)

In [ ]:
client.insert(
    collection_name=COLLECTION_NAME,
    data=data
)

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain.chains.question_answering import load_qa_chain

llmModel = OllamaLLM(model="llama3")
chain=load_qa_chain(llmModel, chain_type="stuff")

In [ ]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    query_vector = embed.embed_query(query)
    print(query_vector)
    matching_results = client.search(
        collection_name=COLLECTION_NAME,
        anns_field="vector",
        data=[query_vector],
        limit=k,
        search_params={"metric_type": "COSINE"},
        output_fields=["content", "metadata"]
    )
    return matching_results

In [ ]:
def retrieve_answers(query):
    doc_search=retrieve_query(query, 4)
    from langchain_core.documents import Document

    documents = []
    for hits in doc_search:
        for hit in hits:
            documents.append(Document(page_content=hit["entity"]["content"], metadata=hit["entity"]["metadata"]))
            #print(json.dumps(hit, indent=4, sort_keys=True))
    
    print(json.dumps(doc_search[0], indent=4, sort_keys=True))
            
    ip = {
        "input_documents":documents,
        "question": query
    }
    response=chain.invoke(input=ip)
    return response

In [ ]:
client.load_collection(
    collection_name=COLLECTION_NAME,
    load_fields=["vector", "id", "content", "metadata"] # Load only the specified fields
)

In [ ]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)


In [ ]:
print("answer: ", type(answer), list(answer.keys()), )
print(answer["output_text"])